# Related Sentences

In [1]:
import pandas as pd
import numpy as np
import spacy
sp_mod = spacy.load("en_core_web_sm")
import os, sys
import glob
from itertools import chain 

In [2]:
def cell_expand(df, lst_cols, fill_value=''):
    # make sure `lst_cols` is a list
    if lst_cols and not isinstance(lst_cols, list):
        lst_cols = [lst_cols]
    # all columns except `lst_cols`
    idx_cols = df.columns.difference(lst_cols)

    # calculate lengths of lists
    lens = df[lst_cols[0]].str.len()

    if (lens > 0).all():
        # ALL lists in cells aren't empty
        return pd.DataFrame({
            col:np.repeat(df[col].values, df[lst_cols[0]].str.len())
            for col in idx_cols
        }).assign(**{col:np.concatenate(df[col].values) for col in lst_cols}) \
          .loc[:, df.columns]
    else:
        # at least one list in cells is empty
        return pd.DataFrame({
            col:np.repeat(df[col].values, df[lst_cols[0]].str.len())
            for col in idx_cols
        }).assign(**{col:np.concatenate(df[col].values) for col in lst_cols}) \
          .append(df.loc[lens==0, idx_cols]).fillna(fill_value) \
          .loc[:, df.columns]

In [3]:
filenames = glob.glob('/Users/Mohit/Desktop/Work/Shreays Dataset/chuncker_raw_new_dataset/cnn/del/*.story', recursive = True)
# filenames = filenames[0]
print(filenames)

['/Users/Mohit/Desktop/Work/Shreays Dataset/chuncker_raw_new_dataset/cnn/del/0a0a4c90d59df9e36ffec4ba306b4f20f3ba4acb.story']


In [4]:
files = {} 
allstory_list = []
for filename in filenames: 
    with open(filename, "r") as file: 
        if filename in files: 
            continue 
        files[filename] = file.read() 
for filename, text in files.items(): 
    allstory_list.append(text)

In [5]:
# allstory_list

In [6]:
def convert_allstory_list(allstory_list):
    final_list = []
    for i in range(len(allstory_list)):
        res_list = allstory_list[0].splitlines()
        res = [ele for ele in res_list if ele != []]
        res = [x for x in res if x]
        # res = res[: len(res) - 8]
        final_list.append(res)
    return final_list

In [7]:
final_list = convert_allstory_list(allstory_list)

In [10]:
# IMP
def convert_pairs(final_list):
    new_list = []
    for i in range(len(final_list)):
        for j in range(len(final_list[i])):
            new_sents = [] # for sentences with lenght greater than 5
            text = final_list[i][j]
            doc = sp_mod(text)
            # Tokenise the sentence
            sents = [elem for elem in doc.sents]

            if len(sents) > 1:

                # Idnetify the length of the sentence
                sents_len = [len(elem) for elem in doc.sents]
                
                # Ignore the tokenise setence with length less than 5 
                for k in range(len(sents_len)):
                    if sents_len[k] > 5:
                        new_sents.append(sents[k])
            else:
                new_sents = []
                
            # Assign to final list
            final_list[i][j] = new_sents

    final_list = list(chain.from_iterable(final_list))
    final_list = [x for x in final_list if x]
    new_list.append(final_list)

    return new_list

In [11]:
final_list_converted_pairs = convert_pairs(final_list)
# final_list_converted_pairs

In [12]:
def pairs_related(new_sents):
    df = pd.DataFrame(columns=['sent_list'], index = range(1))
    df['sent_list'][0] = new_sents

    df['sent_len'] = None
    for i in range(df.shape[0]):
        df['sent_len'][i] = len(df['sent_list'][i])
    df = df[df['sent_len']>1]
    # print("*"*100)
    # print(df.head())
    # print("*"*100)
    df = df.reset_index(drop=True)

    df['sent-1'] = None
    df['sent-2'] = None

    if df['sent_len'][0] > 2:
        for i in range(df.shape[0]):

            sent_1_list = []
            sent_2_list = []
            
            for j in range(df['sent_len'][i]):
                sent_1_list.append(df['sent_list'][i][j])
                if j+1 < df['sent_len'][i]:
                    sent_2_list.append(df['sent_list'][i][j+1])

            df['sent-1'][i] = sent_1_list
            df['sent-2'][i] = sent_2_list
            
            if len(df['sent-1'][i]) > len(df['sent-2'][i]):
                df['sent-1'][i].pop()

        col_list = ['sent-1','sent-2']
        df = cell_expand(df, lst_cols=col_list)
        df = df.reset_index(drop=True)

    else:

        for i in range(df.shape[0]):
            df['sent-1'][i] = df['sent_list'][i][0]
            df['sent-2'][i] = df['sent_list'][i][1]
        df = df.reset_index(drop=True)

    df['sent-1'] = df['sent-1'].astype('str')
    df['sent-2'] = df['sent-2'].astype('str')
    df['sent-1'].replace('', np.nan, inplace=True)
    df.dropna(subset=['sent-1'], inplace=True)
    df['sent-2'].replace('', np.nan, inplace=True)
    df.dropna(subset=['sent-2'], inplace=True)
    df = df[['sent-1','sent-2']]
    final_list2 = df.values.tolist()
    return final_list2

In [13]:
final_list_converted_pairs

[[[Can a movie actually convince you to support torture?,
   Can a movie really persuade you that "fracking" -- a process used to drill for natural gas -- is a danger to the environment?,
   Can a movie truly cause you to view certain minority groups in a negative light?],
  [Some scoff at the notion that movies do anything more than entertain.,
   Sure, it's unlikely that one movie alone will change your views on issues of magnitude.,
   But a movie (or TV show) can begin your "education" or "miseducation" on a topic.,
   And for those already agreeing with the film's thesis, it can further entrench your views.],
  [Anyone who doubts the potential influence that movies can have on public opinion need to look no further than two films that are causing an uproar even before they have opened nationwide.,
   They present hot button issues that manage to fire up people from the left and right.],
  [The first, "Zero Dark Thirty," is about the pursuit and killing of Osama bin Laden, which fe

In [14]:
final_df = pd.DataFrame(columns = ["sent-1", "sent-2"])
for i in range(len(final_list_converted_pairs)):
        for j in range(len(final_list_converted_pairs[i])):
            try:
                inter_df = pd.DataFrame(pairs_related(final_list_converted_pairs[i][j]))
                inter_df.columns = ["sent-1", "sent-2"]
                final_df = final_df.append(inter_df)
            except:
                # print(pairs_related(final_list_converted_pairs[i][j]))
                continue
final_df = final_df.reset_index(drop=True)

<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths o

In [15]:
display(final_df.shape, final_df.head())

(18, 2)

,sent-1,sent-2
0,Can a movie actually convince you to support t...,"Can a movie really persuade you that ""fracking..."
1,"Can a movie really persuade you that ""fracking...",Can a movie truly cause you to view certain mi...
2,Some scoff at the notion that movies do anythi...,"Sure, it's unlikely that one movie alone will ..."
3,"Sure, it's unlikely that one movie alone will ...","But a movie (or TV show) can begin your ""educa..."
4,"But a movie (or TV show) can begin your ""educa...",And for those already agreeing with the film's...


In [16]:
# print(final_list2)